In [1]:
import json
import os
import pysolr
import requests
import pandas as pd

In [2]:
CORE_NAME = "IRF24P1"
VM_IP = "34.127.2.180"

In [3]:
solr = pysolr.Solr(f'http://{VM_IP}:8983/solr/{CORE_NAME}', always_commit=True)

### Delete all documents matching query q

In [13]:
solr.delete(q='*:*')

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">30</int>\n</lst>\n</response>\n'

### Deleting Fields??

#### Couln't delete fields that are referenced by a copy field directive in Solr. A copy field is a mechanism 
#### that copies the value from one field to another field in Solr which is often used for indexing or 
#### search optimization.

### Listing Copy fields

In [61]:
response = requests.get(f'http://{VM_IP}:8983/solr/{CORE_NAME}/schema/copyfields')
print(response.json())

{'responseHeader': {'status': 0, 'QTime': 0}, 'copyFields': [{'source': 'summary', 'dest': 'summary_str', 'maxChars': 256}, {'source': 'topic', 'dest': 'topic_str', 'maxChars': 256}, {'source': 'title', 'dest': 'title_str', 'maxChars': 256}, {'source': 'url', 'dest': 'url_str', 'maxChars': 256}, {'source': '_doc', 'dest': '_doc_str', 'maxChars': 256}]}


### Removing Copy Field directives

In [64]:
data = {
    "delete-copy-field": [
        {
            "source": "summary",
            "dest": "summary_str"
        },
        {
            "source": "topic",
            "dest": "topic_str"
        },
        {
            "source": "url",
            "dest": "url_str"
        },
        {
            "source": "title",
            "dest": "title_str"
        },
        
    ]
}

response = requests.post(f'http://{VM_IP}:8983/solr/{CORE_NAME}/schema', json=data)
print(response.json())

{'responseHeader': {'status': 0, 'QTime': 431}}


## NOW Deleting Fields

In [14]:
data = {
    "delete-field": [
        {"name": "revision_id"},
        {"name": "title"},
        {"name": "topic"},
        {"name": "url"},
        {"name": "summary"}
    ]
}

response = requests.post(f'http://{VM_IP}:8983/solr/{CORE_NAME}/schema', json=data)
print(response.json())

{'responseHeader': {'status': 0, 'QTime': 363}}


## Adding new fields

In [15]:
data = {
            "add-field": [
                {
                    "name": "revision_id",
                    "type": "string"
                },
                {
                    "name": "title",
                    "type": "string"
                },
                {
                    "name": "summary",
                    "type": "text_en"
                },
                {
                    "name": "url",
                    "type": "string"
                },
                {
                    "name": "topic",
                    "type": "string"
                }
            ]
        }

# data = {
#             "add-field": [
#                 {
#                     "name": "revision_id",
#                     "type": "text_en"
#                 },
#                 {
#                     "name": "title",
#                     "type": "text_en"
#                 },
#                 {
#                     "name": "summary",
#                     "type": "text_en"
#                 },
#                 {
#                     "name": "url",
#                     "type": "text_en"
#                 },
#                 {
#                     "name": "topic",
#                     "type": "text_en"
#                 }
#             ]
#         }
requests.post(f'http://{VM_IP}:8983/solr/{CORE_NAME}/schema', json=data).json()

{'responseHeader': {'status': 0, 'QTime': 337}}

## Adding docs from saved json file

In [16]:
df = pd.read_json("document_final.json")
docs = df.to_dict('records')
solr.add(docs)

'{\n  "responseHeader":{\n    "status":0,\n    "QTime":1683}}\n'